In [1]:
import pandas as pd
import tensorflow
import numpy as np
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.applications import MobileNetV2

In [2]:
base_model=MobileNetV2(include_top=False, weights="imagenet", input_shape=(224,224,3))


9406464/9406464 [==============================] - 0s 0us/step


In [3]:
model = Sequential()
model.add(base_model)
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(7, activation='sigmoid'))

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data=pd.read_csv("/content/drive/MyDrive/Projects/RD/RD_Dermatological_Project_3A/Dataset HAM 10k/HAM10000_metadata.csv")
data['image_full_name']=data['image_id']+'.jpg'
X=data[['image_full_name','dx','lesion_id']]

In [8]:
from sklearn.model_selection import train_test_split
Y=X.pop('dx').to_frame()
X_train, X_test, y_train, y_test =  train_test_split(X,Y, test_size=0.17, random_state=42)
X_train, X_val, y_train, y_val   =  train_test_split(X_train, y_train, test_size=0.17, random_state=42)

In [9]:
train=pd.concat([X_train,y_train],axis=1)
val=pd.concat([X_val,y_val],axis=1)
test=pd.concat([X_test,y_test],axis=1)

In [10]:
from sklearn.preprocessing import LabelEncoder
encoder= LabelEncoder()
encoder.fit(val['dx'])
name_as_indexes_train=encoder.transform(val['dx'])
val['label']=name_as_indexes_train

In [11]:
encoder=LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test=encoder.transform(test['dx'])
test['label']=name_as_indexes_test

In [12]:
# from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale = 1./255,
                                     rotation_range=10,
                                     zoom_range = 0.1,
                                     width_shift_range=0.0,  height_shift_range=0.00)

In [13]:
train_data= train_generator.flow_from_dataframe(dataframe=train,x_col="image_full_name",y_col="dx",
                                                batch_size=32,directory="/content/drive/MyDrive/Projects/RD/RD_Dermatological_Project_3A/Dataset HAM 10k/HAM10000_images",
                                                shuffle=True,class_mode="categorical",target_size=(224,224))

Found 6898 validated image filenames belonging to 7 classes.


In [14]:
test_generator=ImageDataGenerator(rescale = 1./255,rotation_range=10,
                                     zoom_range = 0.1,
                                     width_shift_range=0.0,  height_shift_range=0.02)

In [15]:
test_data= test_generator.flow_from_dataframe(dataframe=test,x_col="image_full_name",y_col="dx",
                                              directory="/content/drive/MyDrive/Projects/RD/RD_Dermatological_Project_3A/Dataset HAM 10k/HAM10000_images",
                                              shuffle=False,batch_size=1,class_mode=None,target_size=(224,224))

Found 1703 validated image filenames.


In [16]:
val_data=test_generator.flow_from_dataframe(dataframe=val,x_col="image_full_name",y_col="dx",
                                            directory="/content/drive/MyDrive/Projects/RD/RD_Dermatological_Project_3A/Dataset HAM 10k/HAM10000_images",
                                            batch_size=64,shuffle=False,class_mode="categorical",target_size=(224,224))

Found 1414 validated image filenames belonging to 7 classes.


In [17]:
from keras.callbacks import ReduceLROnPlateau
learning_control = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=.5, min_lr=0.0001)

In [18]:
from keras.optimizers import Adam,SGD
sgd = optimizers.SGD(lr = 0.01, clipvalue = 0.5)
model.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

In [19]:
history = model.fit_generator(generator=train_data,
                            steps_per_epoch=train_data.samples//train_data.batch_size,
                            validation_data=val_data,
                            verbose=1,
                            validation_steps=val_data.samples//val_data.batch_size,
                            epochs=1,  callbacks=[learning_control])

<ipython-input-19-2f04b630ad0a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_data,


215/215 [==============================] - ETA: 0s - loss: 1.0884 - accuracy: 0.6593

215/215 [==============================] - 2426s 11s/step - loss: 1.0884 - accuracy: 0.6593 - val_loss: 1.4524 - val_accuracy: 0.6733 - lr: 0.0100


In [20]:
model.save('modelh110k.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [22]:
test_data.reset()
predictions = model.predict_generator(test_data, steps=test_data.samples/test_data.batch_size,verbose=1)
y_pred= np.argmax(predictions, axis=1)

<ipython-input-22-99bdc2a40673>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(test_data, steps=test_data.samples/test_data.batch_size,verbose=1)


1703/1703 [==============================] - 464s 272ms/step


In [27]:
c=np.where(y_pred == name_as_indexes_test)     #test data is 1703

In [28]:
np.count_nonzero(c)

1128

In [60]:
print("Test Accuracy :",(np.count_nonzero(c)/len(name_as_indexes_test))*100)

# important this is most

Test Accuracy : 66.23605402231357


In [30]:
model.save("modelh110k.h5")

## **Inception V3 model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.applications.inception_v3 import InceptionV3


#Instantiates InceptionV3 architecture
base_model = InceptionV3(weights='imagenet', include_top=False)

base = base_model.output
#Add a global spatial average pooling layer
vectors = tf.keras.layers.GlobalAveragePooling2D()(base)
#Add a fully-connected layer
dense = tf.keras.layers.Dense(512, activation='relu')(vectors)
#Add a logistic layer
predictions = tf.keras.layers.Dense(7, activation='softmax')(dense)

#Build the model
model_2 = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model_2.summary()   #To see how many layers to be frozen

In [ ]:
#Freeze the first 249 layers and unfreeze the rest
for layer in model_2.layers[:249]:
   layer.trainable = False
for layer in model_2.layers[249:]:
   layer.trainable = True

In [ ]:
from tensorflow.keras.optimizers import SGD

model_2.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001,momentum=0.9), metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_control = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=.5, min_lr=0.0001)

In [ ]:
history_2 = model_2.fit_generator(generator=train_data,
                            steps_per_epoch=train_data.samples//train_data.batch_size,
                            validation_data=val_data,
                            verbose=1,
                            validation_steps=val_data.samples//val_data.batch_size,
                            epochs=20,
                            callbacks=[learning_control])

Epoch 1/20


KeyboardInterrupt: 

In [ ]:
# history_2 = model_2.fit(generator=train_data, epochs=25, steps_per_epoch=10, validation_data=val_data,
#                         verbose=1, validation_steps=2, class_weight=class_weights, callbacks=[checkpoint])

NameError: name 'class_weights' is not defined

#<b>Gradio

In [32]:
!pip install gradio

In [33]:
import gradio as gr

In [40]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [62]:
import gradio as gr
import tensorflow as tf
import numpy as np

# Load your saved TensorFlow model (adjust the path if needed)
MODEL_PATH = 'modelh110k.h5'
model = tf.keras.models.load_model(MODEL_PATH)

# Preprocess function (adjust based on your model's input format)
def preprocess_image(image):
    # Resize the image to the model's expected input size
    image_size = (224, 224)  # Assuming MobileNetV2 expects 224x224
    image = tf.image.resize(image, image_size)
    # Normalize pixel values (typical for image classification models)
    image = image / 255.0
    # Add a batch dimension if the model expects it
    return np.expand_dims(image, axis=0)

# Prediction function
def predict_image(image):
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    # Make predictions using the loaded model
    prediction = model.predict(preprocessed_image)

    # Assuming your model has class names corresponding to the output indices
    class_names = ['Melanocytic nevi', 'Melanoma', 'Benign keratosis-like lesions',
                   'Basal cell carcinoma', 'Vascular lesions', 'Actinic keratoses',
                   'Dermatofibroma']

    # Get the class with the highest probability
    predicted_class_index = np.argmax(prediction[0])
    predicted_class = class_names[predicted_class_index]

    # Optionally, return multiple top predictions with probabilities:
    top_3_indices = prediction[0].argsort()[-3:][::-1]
    top_3_probs = prediction[0][top_3_indices] * 100  # Convert to percentages

    # Remember, this is for informational purposes only. Always consult a healthcare professional.
    return f"Predicted Disease:\n  - {predicted_class}\nTop 3 Probabilities:\n   - {class_names[top_3_indices[0]]}: {top_3_probs[0]:.2f}%\n   - {class_names[top_3_indices[1]]}: {top_3_probs[1]:.2f}%\n   - {class_names[top_3_indices[2]]}: {top_3_probs[2]:.2f}%"

# Define Gradio interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.components.Image(label="Skin Lesion Image"),
    outputs=gr.Textbox(label="Predicted skin Disease"),
    title="AI Dermatological Assistance"
)

# Launch the interface
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://014de63dafdeacb7bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
